# ONE advanced queries

While the basic `one.search` command allows us to filter experimental sessions by date, lab, user, subject etc, you may quickly find that you want to apply more complex queries to restrict your search output. This can be achieved using the `one.alyx.rest` command. This has access to a range of tables stored in Alyx and gives us more flexibilty when forming our queries.

## `one.search` vs `one.alyx.rest`
We will get started by importing ONE

In [ ]:
from oneibl.one import ONE
one = ONE()

We can list the possible tables that we have access to, by typing the following command

In [ ]:
one.alyx.rest()

Let's first look at `sessions` table. The information about what filters can be applied to this table are available under **Query Parameters** [here](https://alyx.internationalbrainlab.org/docs/#sessions-list). Let's apply a filter based on `subject` and `task_protocol`. We can do this using the following expression,

In [ ]:
sess_info = one.alyx.rest('sessions', 'list', subject='SWC_015', task_protocol='ephys')
sess_info[0]

Notice how this command doesn't just return the session eID but a dictionary containing information about each session. We can extract the set of eIDs from the `url` key 

In [ ]:
eids = [s['url'] for s in sess_info]
eids = [e.split('/')[-1] for e in eids]
eids

You might have noticed that this same query could be have been achieved using the `one.search` method,

In [ ]:
eids = one.search(subject='SWC_015', task_protocol='ephys')
eids

Let's add in another filter to see the benefits of the `one.alyx.rest` approach. We will add an `atlas_acronym` filter to see if any of the ephys sessions from SWC_015 were recorded from the somatomotor areas (acronym MO). This query would not be possible with the `one.search` command!

In [ ]:
sess_info = one.alyx.rest('sessions', 'list', subject='SWC_015', task_protocol='ephys',
                          atlas_acronym='MO')
sess_info

## Accessing other Alyx tables

With the `one.alyx.rest` command we are not only limited to the `sessions` table, but can formulate queries based on other tables to find session eIDs of interest. Consider the case where we want to find all sessions that have probe insertions that target a specific ML and AP coordinate. For this we can formulate our query based on the [trajectories table](https://alyx.internationalbrainlab.org/docs/#trajectories-list). Let's see if there are any probe insertions at the coordinates ML(x) = -2000, and AP(y) = -2000 from bregma. 

In [ ]:
trajs = one.alyx.rest('trajectories', 'list', x=-2000, y=-2000)
trajs[0]

We can find the session eID associated with this trajectory by looking at the `id` of the `session` field in the returned dictionary

In [ ]:
eid = trajs[0]['session']['id']
eid

<div class="alert alert-info">

Note. 
    
It is not just sessions that have unique IDs associated with them! Every object stored in Alyx has a unique UUID, whether it is a trajectory, a subject, a user or a dataset. For example in the above example we can access the unique ID of the trajectory by typing `traj_id = trajs[0]['id']`


</div>

## Searching with `one.alyx.rest` 
The `one.alyx.rest` command is also a far superior method compared to `one.list` for searching the database for possible keywords that you can use to restrict your queries. For example, if we want to find the names of female subjects in the Witten lab that are alive, we can use the [subjects table](https://alyx.internationalbrainlab.org/docs/#subjects-list) to write,

In [ ]:
subj_info = one.alyx.rest('subjects', 'list', lab='wittenlab', sex='F', alive=True)
subj_nickname = [subj['nickname'] for subj in subj_info]
subj_nickname